# Export onnx with your trained weights

In [1]:
import os
def get_latest_checkpoint(base_dir):
    ckpt_file = os.path.join(base_dir, "last_checkpoint")
    if os.path.exists(ckpt_file):
        with open(ckpt_file, "r") as f:
            relative_ckpt_path = f.readline().strip()
            full_ckpt_path = os.path.join(base_dir, relative_ckpt_path)
            if os.path.exists(full_ckpt_path):
                return full_ckpt_path
    return None

# the logs stored in work_dirs/convnext-v2-tiny_32xb32_in1k-384px_custom/
latest_ckpt = get_latest_checkpoint("mmpretrain/work_dirs/convnext-v2-tiny_32xb32_in1k-384px_custom/")
print(f"Using checkpoint: {latest_ckpt}")

Using checkpoint: /home/test/carasml/classification/mm_Convnext/mmpretrain/work_dirs/convnext-v2-tiny_32xb32_in1k-384px_custom/epoch_10.pth


In [2]:
# revise the code according to your model
# the second line is the location of exporting script, base on your nature of your task (classification/ detection/ segmentation)
# the third line is the location of configs of your model
# the fourth line is the location of weights (pth) of your model
# the fifth line is a sample of image
# the sixth line is exporting location
# the seventh line is choosing model to run on cpu or cuda

In [2]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/convnext-v2-tiny_32xb32_in1k-384px_custom/convnext-v2-tiny_32xb32_in1k-384px_custom.py \
{latest_ckpt} \
mmpretrain/data/ZhangLabData/CellData/OCT/test/NORMAL/NORMAL-1569-1.jpeg \
--work-dir mmdeploy_model/convnext \
--device cpu

07/09 13:27:49 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
07/09 13:27:50 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:27:50 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/09 13:27:50 - mmengine - INFO - Because batch augmentations are enabled, the data preprocessor automatically enables the `to_onehot` op

# Inference the onnx

In [3]:
# providers set provider priority cuda or cpu
import onnxruntime
sess = onnxruntime.InferenceSession("mmdeploy_model/convnext/end2end.onnx", providers=["CUDAExecutionProvider"])
print(sess.get_providers())

['CUDAExecutionProvider', 'CPUExecutionProvider']


In [5]:
'''
from mmdeploy.apis import inference_model
is the official method to run an onnx, but it require torch
'''
#Classification
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/convnext-v2-tiny_32xb32_in1k-384px_custom/convnext-v2-tiny_32xb32_in1k-384px_custom.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/convnext/end2end.onnx'],
    img='mmpretrain/data/ZhangLabData/CellData/OCT/test/NORMAL/NORMAL-1569-1.jpeg',
    device='cpu')
print(result)

[<DataSample(

META INFORMATION
    img_path: mmpretrain/data/ZhangLabData/CellData/OCT/test/NORMAL/NORMAL-1569-1.jpeg
    ori_shape: (496, 1024)
    num_classes: 4
    img_shape: (384, 384)
    scale_factor: (0.375, 0.7741935483870968)

DATA FIELDS
    pred_score: tensor([0.0791, 0.0510, 0.0577, 0.8122])
    pred_label: tensor([3])

) at 0x7410b06c3040>]


In [6]:
'''
this is an unofficial method to run an onnx that require no torch
however, you need to check the 'model_cfg' to understand how the image was preprocessed
which include normalization method and resize and padding method
you may search keywords "data_preprocessor" and "mean" and "std" for normalization 
and "pipeline" and "scale" for resize and padding method
'''
#Classification 
import onnxruntime
import numpy as np
from PIL import Image

# we resize
def resize(image, target_size):
    # Resize the image with aspect ratio preserved
    image = image.resize((target_size, target_size), Image.BICUBIC)

    return image
    
# Load the ONNX model with cpu or cuda execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/convnext/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/data/ZhangLabData/CellData/OCT/test/NORMAL/NORMAL-1569-1.jpeg"

# Load and preprocess the image
image = Image.open(image_path)
image = resize(image, target_size=384)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.array(image)
input_array = np.expand_dims(input_array,axis=0)
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            123.675,
            116.28,
            103.53,
        ],
        num_classes=4,
        std=[
            58.395,
            57.12,
            57.375,
        ],
        to_rgb=True)
'''
#normalize image
mean = np.array([123.675, 116.28, 103.53]).reshape(1, 3, 1, 1) 
std = np.array([58.395, 57.12, 57.375]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

# Print the output
print("Model output:", output)


Available providers: ['CPUExecutionProvider']
Model output: [[0.07913636 0.05096158 0.0577339  0.81216806]]
